# <p style="text-align: center;">Проект по SQL: Сервис для чтения книг</p>

### Описание задачи SQL:
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных:
Таблица books - Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

Таблица authors - Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

Таблица publishers - Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

Таблица ratings - Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

Таблица reviews - Содержит данные о пользовательских обзорах на книги:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя пользователя, написавшего обзор;
- text — текст обзора.

**ER-диаграмма:**
- Структура данных БД

![Изображение](https://2.downloader.disk.yandex.ru/preview/f18a76e943e2a5034b764c67d71438ed5dd9d53a0dee46535314c34e89154222/inf/AJISjF4h-BPUuZeyyyQZsDYA0hPpw8437gQdx0o8XGbiT5GuN_HjXhr7xaLd_PbhdU6GUWwiRbfDd1J6kRtp8Q%3D%3D?uid=375844755&filename=ER-dia.jpg&disposition=inline&hash=&limit=0&content_type=image%2Fjpeg&owner_uid=375844755&tknv=v2&size=910x942 "ER-диаграмма")

### Задачи:
- Посчитаем, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитаем количество обзоров и среднюю оценку;
- Определим издательство, которое выпустило наибольшее число книг толще 50 страниц  — так мы исключите из анализа брошюры;
- Определим автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
- Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


### Подключение к базе данных

In [1]:
# установим драйвер PostgreSQL
!pip install psycopg2

     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.2 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.2 MB 907.3 kB/s eta 0:00:02
     -- ------------------------------------- 0.1/1.2 MB 907.3 kB/s eta 0:00:02
     ------ --------------------------------- 0.2/1.2 MB 1.2 MB/s eta 0:00:01
     ----------- ---------------------------- 0.3/1.2 MB 1.4 MB/s eta 0:00:01
     -------------- ------------------------- 0.4/1.2 MB 1.7 MB/s eta 0:00:01
     ------------------------ --------------- 0.7/1.2 MB 2.2 MB/s eta 0:00:01
     ------------------------------ --------- 0.9/1.2 MB 2.6 MB/s eta 0:00:01
     ---------------------------------------  1.2/1.2 MB 2.8 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 2.5 MB/s eta 0:00:00


In [2]:
# импортируем библиотеки
import pandas as pd
# Библиотека для выполнения запросов к БД
import sqlalchemy as sa
import psycopg2
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
#Открываем соединение, получаем данные из sql, закрываем соединение
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

### Исследование таблиц

Выведим первые строки, посчитайте количество строк в каждой таблице

In [3]:
# Посмотрим на таблицу с информацией о книгах
# и подсчитаем кол-во строк в таблице используя для этого оконную функцию.
# выведем первую строку для исседования столбцов
query = '''
           SELECT *, COUNT(book_id) OVER() cnt_line
           FROM books
           LIMIT 1;
           '''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id,cnt_line
0,1,546,'Salem's Lot,594,2005-11-01,93,1000


В данной таблице с информацией о книгах всего 1000 строк.

In [4]:
# Посмотрим на таблицу с информацией о авторах и подсчитаем кол-во строк в таблице.
query = '''
           SELECT *, COUNT(author_id) OVER() cnt_line
           FROM authors
           LIMIT 1;
           '''
get_sql_data(query)

,author_id,author,cnt_line
0,1,A.S. Byatt,636


В данной таблице с информацией о авторах всего 636 строк.

In [5]:
# Посмотрим на таблицу с информацией о издательствах и подсчитаем кол-во строк в таблице.
query = '''
           SELECT *, COUNT(publisher_id) OVER() cnt_line
           FROM publishers
           LIMIT 1;
           '''
get_sql_data(query)

,publisher_id,publisher,cnt_line
0,1,Ace,340


В данной таблице с информацией о издательствах всего 340 строк.

In [6]:
# Посмотрим на таблицу с информацией о пользовательских оценках книг
# и подсчитаем кол-во строк в таблице.
query = '''
           SELECT *, COUNT(rating_id) OVER() cnt_line
           FROM ratings
           LIMIT 1;
           '''
get_sql_data(query)

,rating_id,book_id,username,rating,cnt_line
0,1,1,ryanfranco,4,6456


В данной таблице с информацией о пользовательских оценках книг всего 6456 строк.

In [7]:
# Посмотрим на таблицу с информацией пользовательских обзорах на книги
# и подсчитаем кол-во строк в таблице.
query = '''
           SELECT *, COUNT(review_id) OVER() cnt_line
           FROM reviews
           LIMIT 1;
           '''
get_sql_data(query)

,review_id,book_id,username,text,cnt_line
0,1,1,brandtandrea,Mention society tell send professor analysis. ...,2793


В данной таблице с информацией пользовательских обзорах на книги всего 2793 строки.

### Решение задач:

In [8]:
# Посчитайте, сколько книг вышло после 1 января 2000 года
query = '''
           SELECT COUNT(book_id) cnt_books
           FROM books
           WHERE publication_date > '2000-01-01'
           '''
get_sql_data(query)

,cnt_books
0,819


После 1 января 2000 года вышло 819 книг.

In [9]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку;
# Для подсчета кол-ва обзоров и средней оценки нужно выполнить соединение трех таблиц по ключу
# группируем по названию книги и используем функции агрегации к нужным столбцам.
# Так как есть обзоры разных авторов,
# то нужны уникальные  идентификаторы обзора используем distinct при подсчете кол-ва обзоров на книгу
query = '''
           SELECT b.title,
                  b.book_id,
                  COUNT(DISTINCT rw.review_id) review,
                  AVG(rt.rating) avg_rating
           FROM books b JOIN ratings rt
           ON b.book_id = rt.book_id
           LEFT JOIN reviews rw
           ON b.book_id = rw.book_id
           GROUP BY b.title, b.book_id
           ORDER BY review, avg_rating
           '''
get_sql_data(query)
# Отсортировал по кол-ву обзоров и рейтингу для наглядности.

,title,book_id,review,avg_rating
0,The Natural Way to Draw,808,0,3.000000
1,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.666667
2,Leonardo's Notebooks,387,0,4.000000
3,Essential Tales and Poems,221,0,4.000000
4,Disney's Beauty and the Beast (A Little Golden...,191,0,4.000000
...,...,...,...,...
995,The Glass Castle,734,6,4.206897
996,The Book Thief,656,6,4.264151
997,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.287500
998,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.414634


В данных имеются книги, которые не имеют обзоров, но при этом есть рейтинг, при подсчете кол-ва обзоров на книгу брали уникальных авторов обзоров, также подсчитали средние оценки для каждой книги.

In [10]:
# Данная книга имеет дубликат в данных
query = '''
           SELECT title, COUNT(title)
           FROM books 
           GROUP BY title
           HAVING COUNT(title) > 1
           '''
get_sql_data(query)

,title,count
0,Memoirs of a Geisha,2


У книги 'Memoirs of a Geisha' в данных имеется дубликат. Выясним почему.

In [11]:
# Отобразим издательства
query = '''
           SELECT b.title,
                  b.book_id,
                  p.publisher,
                  COUNT(DISTINCT rw.review_id) review,
                  AVG(rt.rating) avg_rating
           FROM books b JOIN ratings rt
           ON b.book_id = rt.book_id
           LEFT JOIN reviews rw
           ON b.book_id = rw.book_id
           JOIN publishers p
           ON b.publisher_id = p.publisher_id
           WHERE b.title LIKE '%Memoirs of a Geisha%'
           GROUP BY b.title, b.book_id, p.publisher
           ORDER BY review, avg_rating
           '''
get_sql_data(query)

,title,book_id,publisher,review,avg_rating
0,Memoirs of a Geisha,427,Vintage Books USA,3,3.700000
1,Memoirs of a Geisha,426,Random House Large Print Publishing,5,4.195652


Видим, что у данная книга выпускалась под двумя разными издательствами.

In [12]:
# Определите издательство, которое выпустило наибольшее число книг толще
# 50 страниц — так вы исключите из анализа брошюры
# Соединим таблицу с книгами и издательствами и
# зададим условие, где число страниц больше 50 и подсчитаем число книг сруппированные по издательству,
# выполним сортировку по убыванию и выберим первое значение.
query = '''
           SELECT COUNT(book_id) cnt_books, 
                  pb.publisher
           FROM books b JOIN publishers pb
           ON b.publisher_id = pb.publisher_id 
           WHERE num_pages > 50
           GROUP BY pb.publisher
           ORDER BY cnt_books DESC
           LIMIT 1;
           '''
get_sql_data(query)

,cnt_books,publisher
0,42,Penguin Books


Издательство выпустившее наибольшее число книг толщиной более 50 страниц является 'Penguin Books'.

In [13]:
# Определите автора с самой высокой средней оценкой книг - учитываем только книги с 50 и более оценками;
# Выполним соединение трех таблиц (книги, авторы, рейтинги),
# в условии отберем те книги, которые с 50 и более оценками (подзапросом),
# в основном запросе сгруппируем средние оценки по авторам и отсортируем  по убыванию,
# выберем первую строку отражающую автора с самой высокой средней оценкой.
query = '''
           SELECT a.author, AVG(rating) rating
           FROM books b JOIN authors a
           ON b.author_id = a.author_id
           JOIN ratings rt 
           ON b.book_id = rt.book_id
           WHERE b.book_id IN (SELECT book_id
                               FROM ratings
                               GROUP BY book_id
                               HAVING COUNT(rating_id) >= 50)
           GROUP BY a.author
           ORDER BY rating DESC
           LIMIT 1
           '''
get_sql_data(query)

,author,rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг является - 'J.K. Rowling/Mary GrandPré'.

In [14]:
# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
query = '''
            SELECT AVG(x.review) avg_cnt_review
            FROM (SELECT COUNT(review_id) review
                  FROM reviews
                  WHERE username IN (SELECT username
                                     FROM ratings
                                     GROUP BY username
                                     HAVING COUNT(rating) > 48)
                  GROUP BY username
                  ) x
           
        '''
get_sql_data(query)

,avg_cnt_review
0,24.0


Среднее количество обзоров от пользователей, которые поставили
больше 48 оценок в среднем равно 24 обзорам.

### Вывод:

Проанализировав базу данных сервиса для чтения книг определили:

    - После 1 января 2000 года вышло 819 книга;
    - Количество обзоров и среднюю оценку для каждой книги;
    - Нашли книгу под двумя разными издательствами;
    - Издательство выпустившее наибольшее число книг толщиной более 50 страниц, которая является 'Penguin Books';
    - Автора с самой высокой средней оценкой книг - 'J.K. Rowling/Mary GrandPré';
    - Что в среднем 24 обзора от пользователей, которые поставили больше 48 оценок. Стоит обратить на это внимание, нужно ввести поощрение пользователям, которые оставляют рецензии.
Рекомендации:

- Продвигать новые книги, выпущенные после 1 января 2000 года: Это позволит удовлетворить интересы пользователей к современной литературе и обеспечить им актуальные книги.
- Книги, выпущенные под разными издательствами, возможно, это связанно с дизайном книги или ее качеством, переводами или другими факторами, которые могут быть интересны пользователям.